## This notebook specifically targets the binary classifier, i.e. to predict if it is an attack or a benign case.
**Sujal Adhikari**

In [2]:
### Libraries that we will be using:

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression


In [6]:
### Dataset we will be using 
data = pd.read_csv('../Datasets/pre-processedData.csv')

## We will be excluding the column named 'Severity_Level'ArithmeticError
binary_data = data.drop(columns=['is_malicious', 'Unnamed: 0'])
binary_data.head(5)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,...,flag_REJ,flag_RSTO,flag_RSTR,flag_S1,flag_S3,flag_SF,flag_SH,Severity_Score,bytes_ratio,total_bytes
0,0.0,6.198479,0.000000,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,1,0,0.0,6.198479,6.198479
1,0.0,4.990433,0.000000,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,1,0,0.0,4.990433,4.990433
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,2.0,0.000000,0.000000
3,0.0,5.451038,9.006264,0.0,0.0,0.0,0.0,1,0.0,0,...,0,0,0,0,0,1,0,0.0,0.544763,14.457302
4,0.0,5.298317,6.042633,0.0,0.0,0.0,0.0,1,0.0,0,...,0,0,0,0,0,1,0,0.0,0.752321,11.340950


### Test-Train Split